In [ ]:
import os
import glob
import pandas as pd
import geopandas as gpd
import pickle
import param as pm
import panel as pn
pn.extension('echarts')
import numpy as np
import xarray as xr
import rioxarray as riox
import geoviews as gv
import holoviews as hv
import hvplot.xarray
from copy import deepcopy
from holoviews import streams
import affine
from bokeh.models.formatters import PrintfTickFormatter
from bokeh.models import NumeralTickFormatter, FixedTicker
from holoviews.plotting.util import process_cmap
from datetime import datetime, timedelta
from pyproj import Proj, transform
from shapely.geometry import Polygon
from src.hls_funcs import fetch
from src.hls_funcs.masks import mask_hls, shp2mask, bolton_mask
from src.hls_funcs.indices import ndvi_func
from src.hls_funcs.smooth import smooth_xr, despike_ts_xr
from src.hls_funcs.predict import pred_bm, pred_cov, pred_bm_se, pred_bm_thresh, xr_cdf

pn.param.ParamMethod.loading_indicator = True

In [ ]:
"""
Get the server base url
"""
"""
Get the server base url
"""
from jupyter_server import serverapp
 
try:
    jupServer = [x for x in serverapp.list_running_servers()][0]
except IndexError:
    jupServer = {'base_url': '/node/ceres19-compute-31-eth.scinet.local/35403/'}

In [ ]:
cper_f = os.path.join('data/ground/cper_pastures_2017_clip.shp')
tifDIR = '/90daydata/cper_neon_aop/hls_nrt/cper/'
prefix = 'cper'

In [ ]:
cper = gpd.read_file(cper_f)
cper_bbox = cper.buffer(2000).to_crs(epsg=3857).total_bounds

In [ ]:
cluster_loc = 'local'
if cluster_loc is None:
    aws=True
elif cluster_loc == 'local':
    print('   setting up Local cluster...')
    from dask.distributed import LocalCluster, Client
    import dask
    cluster = LocalCluster(n_workers=2, threads_per_worker=1)
    client = Client(cluster)
    #display(client)

In [ ]:
from bokeh.models import WheelZoomTool
wheel_zoom = WheelZoomTool(zoom_on_axis=False)

In [ ]:
yr = 2022
da_bare = riox.open_rasterio(os.path.join(tifDIR,
                                         'hls_cover',
                                          prefix + '_hls_BARE_' + 
                                          str(yr) + '.tif'))
da_green = riox.open_rasterio(os.path.join(tifDIR,
                                         'hls_cover',
                                          prefix + '_hls_GREEN_' + 
                                          str(yr) + '.tif'))
da_sd = riox.open_rasterio(os.path.join(tifDIR,
                                         'hls_cover',
                                          prefix + '_hls_SD_' + 
                                          str(yr) + '.tif'))
da_litt = riox.open_rasterio(os.path.join(tifDIR,
                                         'hls_cover',
                                          prefix + '_hls_LITT_' + 
                                          str(yr) + '.tif'))
da_bm = riox.open_rasterio(os.path.join(tifDIR,
                                        'hls_biomass',
                                        prefix + '_hls_bm_' +
                                        str(yr) + '.tif'))
da_bm_se = riox.open_rasterio(os.path.join(tifDIR,
                                        'hls_biomass',
                                        prefix + '_hls_bm_se_' +
                                        str(yr) + '.tif'))
da_ndvi = riox.open_rasterio(os.path.join(tifDIR,
                                        'hls_ndvi',
                                        prefix + '_hls_ndvi_' +
                                        str(yr) + '.tif'))
da_bare.name = 'BARE'
da_green.name = 'GREEN'
da_sd.name = 'SD'
da_litt.name = 'LITT'
da_bm.name = 'Biomass'
da_bm_se.name = 'Biomass_SE'
da_ndvi.name = 'NDVI'

In [ ]:
da_bare = da_bare.where(da_bare > 0.0, other=0.0)
da_bare = da_bare.where(da_bare < 1.0, other=1.0)

da_green = da_green.where(da_green > 0.0, other=0.0)
da_green = da_green.where(da_green < 1.0, other=1.0)

da_sd = da_sd.where(da_sd > 0.0, other=0.0)
da_sd = da_sd.where(da_sd < 1.0, other=1.0)

da_litt = da_litt.where(da_litt > 0.0, other=0.0)
da_litt = da_litt.where(da_litt < 1.0, other=1.0)

cov_mask = da_bare.notnull() & da_green.notnull() & da_sd.notnull() & da_litt.notnull()

In [ ]:
ds = xr.merge([da_bare.where(cov_mask), 
               da_green.where(cov_mask), 
               da_sd.where(cov_mask),
               da_litt.where(cov_mask),
               da_bm.where(da_bm >= 0),
               da_bm_se.where(da_bm >= 0),
               da_ndvi.where((da_ndvi >= 0) & (da_ndvi <= 1))]).rename(
    {'band': 'date'}).chunk(
    {'date': -1, 'y': 50, 'x': 50}).rio.write_crs(
    "epsg:32613", inplace=True).rio.reproject("EPSG:3857")
ds = ds.where(ds != -9999.)

In [ ]:
ds['date'] = [datetime(yr, 1, 1) + timedelta(days=int(x)-1) for x in ds['date'].values]
ds['date'] = ds['date'].dt.date
ds['DRY'] = ds['SD'] + ds['LITT']

In [ ]:
class getData(pm.Parameterized):
    control_width = 150
    stats_width = 380
    stats_margin = (5, 1)
    
    viz_sel = pn.widgets.RadioButtonGroup(options=['Basemap', 'Cover', 'Greenness (NDVI)', 'Biomass', 'Biomass probability'],
                                          value='Cover', orientation='vertical', align='start', width=control_width)

    def viz_callback(target, event):
        if event.new == target.name:
            target.value = True
        else:
            target.value = False

    cov_viz = pn.widgets.Checkbox(name='Cover', value=True)
    bm_viz = pn.widgets.Checkbox(name='Biomass', value=False)
    thresh_viz = pn.widgets.Checkbox(name='Biomass probability', value=False)
    ndvi_viz = pn.widgets.Checkbox(name='Greenness (NDVI)', value=False)

    viz_sel.link(cov_viz, callbacks={'value': viz_callback})
    viz_sel.link(bm_viz, callbacks={'value': viz_callback})
    viz_sel.link(thresh_viz, callbacks={'value': viz_callback})
    viz_sel.link(ndvi_viz, callbacks={'value': viz_callback})
    
    year_picker = pn.widgets.Select(name='Select year', options=list(range(2016, 2023, 1)), value=2022,
                                   width=control_width)
    thresh_picker = pn.widgets.IntSlider(name='Biomass threshold', start=200, end=2000, step=50, value=500,
                                 format=PrintfTickFormatter(format='%d kg/ha'), width=control_width)
    
    data_dict = {'date_range': [str(datetime(year_picker.value, 1, 1).date()),
                                str(datetime(year_picker.value, 12, 31).date())]}
 
    cov_dict = {'R': 'Dry veg',
                'G': 'Green veg',
                'B': 'Bare ground'}

    chunks = {'date': 1, 'y': 250, 'x': 250}
    map_args = dict(rasterize=True, project=False, dynamic=True, xticks=None, yticks=None)
    base_opts = dict(backend='bokeh', xaxis=None, yaxis=None, width=900, height=700,
                         padding=0, active_tools=['pan', wheel_zoom], toolbar='left')
    map_opts = dict(responsive=False, xaxis=None, yaxis=None, width=900, height=700,
                     padding=0, tools=['pan', wheel_zoom, 'box_zoom'],
                     active_tools=['pan', wheel_zoom], toolbar='left')
    
    poly_opts = dict(fill_color=['', ''], fill_alpha=[0.0, 0.0], line_color=['#1b9e77', '#d95f02'],
                 line_width=[3, 3])    
    bg_col='#ffffff'

    css = '''
    .bk.box1 {
      background: #ffffff;
      border-radius: 5px;
      border: 1px black solid;
    }
    '''
    
    date = pn.widgets.DatePicker(name='Select date', width=100)
    
    #date = pm.CalendarDate()
      
    tiles = hv.element.tiles.EsriImagery().opts(**base_opts, level='glyph',
                                                                   xlim=(cper_bbox[0], cper_bbox[2]),
                                                                   ylim=(cper_bbox[1], cper_bbox[3]))
    labels = hv.element.tiles.EsriReference().opts(**base_opts, level='overlay')
    
    base_rng = hv.streams.RangeXY(source=tiles,
                                  x_range = (cper_bbox[0], cper_bbox[2]),
                                  y_range = (cper_bbox[1], cper_bbox[3]))
       
    #basemap = hv.element.tiles.EsriImagery().opts(**map_opts, level='glyph')
        
    polys = hv.Polygons([])

    max_polys = 1
    
    poly_stream = streams.PolyDraw(source=polys, drag=True, num_objects=max_polys,
                                    show_vertices=True, styles=poly_opts)
    edit_stream = streams.PolyEdit(source=polys, shared=True)
    
    bm_cmin = 100
    bm_cmax = 1000
    
    cbar_dict = {
    'Cover': {'cmap': 'brg',
              'clim': (0, 1.0),
              'colorbar_opts': {'height': 20,
                                'width': int(control_width*0.95),
                                'ticker': FixedTicker(ticks=[0.05, 0.50, 0.95]),
                                'major_label_overrides': {0.05: 'Bare', 0.50: 'Dry', 0.95: 'Green'}}},
    'Greenness (NDVI)': {'cmap': 'Viridis',
              'clim': (0.0, 0.5),
              'colorbar_opts': {'height': 20,
                                'width': int(control_width*0.95),
                                'ticker': FixedTicker(ticks=[0.025, 0.175, 0.325, 0.475]),
                                'major_label_overrides': {0.025: 'V. Low', 0.175: 'Low', 0.325: 'Mod', 0.475: 'High'}}},
    'Biomass': {'cmap': 'Inferno',
                'clim': (100, 1500),
                'colorbar_opts': {'height': 20,
                                  'width': int(control_width*0.95),
                                  'ticker': FixedTicker(ticks=np.arange(250, 1750, 250)),
                                  'major_label_overrides': {1500: '1500+'}}},
    'Biomass probability': {'cmap': 'Spectral_r',
              'clim': (0.0, 1.0),
              'colorbar_opts': {'height': 20,
                                'width': int(control_width*0.95),
                                'ticker': FixedTicker(ticks=np.round(np.arange(0.1, 1.1, 0.2), 1)),
                                'major_label_overrides': {0.1: 'Unlikely', 0.3: 'Low',
                                                          0.5: 'Possible', 0.7: 'Likely', 0.9: 'High'}}}
    }
    
    bm_plotrange = np.arange(0, 2500, 50)
    bm_colors = np.array(process_cmap('Inferno', ncolors=len(bm_plotrange)))
    bm_colors[bm_plotrange < bm_cmin] = bm_colors[0]
    bm_colors[bm_plotrange > bm_cmax] = bm_colors[-1]
    bm_colors[(bm_plotrange >= bm_cmin) & 
              (bm_plotrange <= bm_cmax)] = process_cmap('Inferno', ncolors=len(np.arange(bm_cmin, bm_cmax+50, 50)))

    thresh_plotrange = np.arange(0, 1.2, 0.2)
    thresh_colors = process_cmap('Spectral_r', ncolors=len(thresh_plotrange)-1)
    thresh_labels = ['Unlikely', 'Low', 'Possible', 'Likely', 'High']
    
    bm_gauge = pn.indicators.Gauge(
        name='Biomass', bounds=(0, 2500), format='{value} kg/ha', tooltip_format='{b} : {c} kg/ha',
        colors=[(0.20, '#FF6E76'), (0.40, '#FDDD60'), (0.60, '#7CFFB2'), (1, '#58D9F9')],
        num_splits=5, align='center', title_size=12,
        start_angle=180, end_angle=0,
        height=200, width=stats_width,
        value=0, margin=stats_margin)
    
    bm_hist = {
        'title': {
            'text': 'Biomass variability',
            'left': "center"},
        'tooltip': {},
        'legend': {},
        'grid': {'show': False},
        'xAxis': {
            'type': 'value',
            'name': 'Biomass (kg/ha)',
            'nameLocation': 'middle',
            'nameGap': 30,
            'splitLine': {
                'show': False}},
        'yAxis': {'max': 35,
                  'axisLabel': {'formatter': '{value} %'},
                  'splitLine': {
                      'show': False}},
        'series': [{
            'name': "",
            'type': "bar",
            'data': [{'value': [x, 0]} for idx, x in enumerate(
                bm_plotrange)],
            'colorBy': "data"}]}
    bm_echart = pn.pane.ECharts(bm_hist, height=250, width=stats_width, margin=stats_margin)
    
    thresh_bar = {
        'title': {
            'text': "Biomass threshold",
            'left': "center"},
        'tooltip': {},
        'legend': {},
    'grid': {'left': '20%'},
    'xAxis': {
        'type': 'category',
        'name': 'Probability of biomass less than ' + str(thresh_picker.value) + 'kg/ha',
        'nameLocation': 'middle',
        'nameGap': 30,
    'data': thresh_labels},
    'yAxis': {'max': 1000,
              'axisLabel': {'formatter': '{value} ha'}},
    'series': [{
        'name': "",
        'type': "bar",
        'data': [0 for x in thresh_labels],
        'colorBy': "data"
    }],
}
    thresh_echart = pn.pane.ECharts(thresh_bar, height=250, width=stats_width, margin=stats_margin)
    
    cov_pie = {
        'title': {
            'text': "Cover",
            'subtext': "Fractional vegetation cover (%)",
            'left': "center"},
        'tooltip': {'show': True,
                    'formatter': '{b}: {d} %'}, 
        'series': [{
            'type': 'pie',
            'data': [{'value': 0, 'name': 'Litter'},
                     {'value': 0, 'name': 'Bare ground'},
                     {'value': 0, 'name': 'Green veg'},
                     {'value': 0, 'name': 'Dry veg'}],
            'color': ['#ee6666', '#91cc75', '#5470c6', '#fac858'],
            'roseType': 'radius',
            'radius': ["0%", "45%"],
            'label': {
                'edgeDistance': "1%",
                'bleedMargin': 10,
                'alignTo': "edge"},
            'labelLine': {'length': 5}}]}
    
    cov_echart = pn.pane.ECharts(cov_pie, height=250, width=stats_width, margin=stats_margin)
    
    ts_bm = {
        'title': [{'left': 'center', 'text': 'Biomass'}],
        'legend': {'orient': 'vertical',
                                   'right': 10,
                                   'top': 'center'
                                  },
        'tooltip': {'trigger': 'axis'},
        'xAxis': [{'data': ds['date'].astype('str').values}],
        'yAxis': [{}],
        'series': [{'name': 'Region avg.',
                    'type': 'line',
                    'showSymbol': False,
                    'data': ds.mean(['y', 'x'])['Biomass'].astype('int').values,
                    'itemStyle': {'color': 'rgb(0, 0, 0)'}}]}    
    ts_bm_echart = pn.pane.ECharts(ts_bm, height=250, width=stats_width, margin=stats_margin)
    
    ts_ndvi = {
        'title': [{'left': 'center', 'text': 'Greenness (NDVI)'}],
        'tooltip': {'trigger': 'axis'},
        'xAxis': [{'data': ds['date'].astype('str').values}],
        'yAxis': [{}],
        'series': [{'name': 'Region avg.',
                    'type': 'line',
                    'showSymbol': False,
                    'data': ds.mean(['y', 'x'])['NDVI'].values,
                    'itemStyle': {'color': 'rgb(0, 0, 0)'}}]}    
    ts_ndvi_echart = pn.pane.ECharts(ts_ndvi, height=250, width=stats_width, margin=stats_margin)

    
    def __init__(self, **params):
        super(getData, self).__init__(**params)
        self.ds = ds

        self.all_maps = None
        self.cov_map = None
        self.bm_map = None
        self.thresh_map = None
        
        self.cov_stats = None
        self.bm_stats = None
        self.thresh_stats = None
        self.stats_titles = None
        self.ts_bm = None
        
        self.date.value = pd.to_datetime(self.ds['date'].values[-1]).date()
                                        
        self.poly_stream.add_subscriber(self.update_stats)
        
        self.map_init = self.tiles * self.labels
        
        self.controls = pn.Column(pn.Spacer(height=30),
                                  self.viz_sel,
                                  pn.Spacer(height=10),
                                  self.thresh_picker,
                                  pn.Spacer(height=20),
                                  pn.pane.Markdown('*Map legend*', background='white'),
                                  self.update_colorbar, height=550, width=int(self.control_width*1.5))
        
        self.view = self._create_view()
    
    def make_colorbar(self, key, orientation = 'horizontal', position = 'top', colorbar_opts = {}, **kwargs):
        ## create an invisible hv.Heatmap plot just to use its colorbar
        cmap = self.cbar_dict[key]['cmap']
        clim = self.cbar_dict[key]['clim']
        colorbar_opts = self.cbar_dict[key]['colorbar_opts']
        
        hm = hv.HeatMap([(0, 0, clim[0]), (0, 1, clim[1])])
        kwargs.update(dict(colorbar=True,
                           colorbar_opts=colorbar_opts, 
                           clim=clim,
                           alpha=0,
                           show_frame=False,
                           frame_height=0,
                           colorbar_position=position, 
                           toolbar="disable",
                           axiswise=True, framewise=True, shared_axes=False,
                           margin=(0,0),
                           cmap=cmap))
        return hm.opts(xaxis = None, yaxis = None, **kwargs)
      
    @pm.depends('date.value', 'thresh_picker.value')
    def create_maps(self):
        if self.ds is not None:
            if self.edit_stream.data is not None:
                self.polys = self.edit_stream.element
                self.poly_stream = streams.PolyDraw(source=self.polys, drag=True, num_objects=self.max_polys,
                                                    show_vertices=True, styles=self.poly_opts)
                self.edit_stream = streams.PolyEdit(source=self.polys, shared=True)
            elif self.poly_stream.data is not None:
                self.polys = self.poly_stream.element
                self.poly_stream = streams.PolyDraw(source=self.polys, drag=True, num_objects=self.max_polys,
                               show_vertices=True, styles=self.poly_opts)
                self.edit_stream = streams.PolyEdit(source=self.polys, shared=True)
            else:
                self.polys = self.polys
            self.poly_stream.add_subscriber(self.update_stats)

            da_thresh_pre = (np.log(self.thresh_picker.value) - 
                             np.log(ds['Biomass'].sel(date=self.date.value))) / ds['Biomass_SE'].sel(date=self.date.value)
            da_thresh = xr_cdf(da_thresh_pre)
            da_thresh.name = 'Threshold'

            self.cov_map = self.ds[['DRY', 'GREEN', 'BARE']].sel(
                date=self.date.value).to_array().hvplot.rgb(x='x', y='y', 
                                                            bands='variable',
                                                            **self.map_args).opts(**self.map_opts)
            self.bm_map = self.ds['Biomass'].sel(date=self.date.value).hvplot(x='x', y='y',
                                       cmap=self.cbar_dict['Biomass']['cmap'],
                                       clim=self.cbar_dict['Biomass']['clim'],
                                       colorbar=False,
                                       **self.map_args).opts(**self.map_opts)
            self.thresh_map = da_thresh.hvplot(x='x', y='y',
                                               cmap=self.cbar_dict['Biomass probability']['cmap'],
                                               clim=self.cbar_dict['Biomass probability']['clim'], 
                                               colorbar=False,
                                               **self.map_args).options(color_levels=5).opts(**self.map_opts)

            self.ndvi_map = ds['NDVI'].sel(date=self.date.value).hvplot(x='x', y='y',
                                           cmap=self.cbar_dict['Greenness (NDVI)']['cmap'],
                                           clim=self.cbar_dict['Greenness (NDVI)']['clim'],
                                           colorbar=False,
                                           **self.map_args).opts(**self.map_opts)

            #self.all_maps = pn.Tabs(
            #    ('Cover', self.cov_map),
            #    ('Biomass', self.bm_map),
            #    ('Biomass threshold', self.thresh_map), 
            #    ('Greenness (NDVI)', self.ndvi_map))

            if self.base_rng.x_range is not None:
                self.all_maps = pn.Pane(self.tiles.apply.opts(xlim=self.base_rng.param.x_range,
                                                        ylim=self.base_rng.param.y_range)  * 
                                        self.labels.apply.opts(xlim=self.base_rng.param.x_range,
                                                               ylim=self.base_rng.param.y_range) * 
                                        self.cov_map.apply.opts(visible=self.cov_viz.param.value, 
                                                                xlim=self.base_rng.param.x_range,
                                                                ylim=self.base_rng.param.y_range) *
                                        self.ndvi_map.apply.opts(visible=self.ndvi_viz.param.value, 
                                                                 xlim=self.base_rng.param.x_range,
                                                                 ylim=self.base_rng.param.y_range) *
                                        self.bm_map.apply.opts(visible=self.bm_viz.param.value, 
                                                               xlim=self.base_rng.param.x_range,
                                                               ylim=self.base_rng.param.y_range) *
                                        self.thresh_map.apply.opts(visible=self.thresh_viz.param.value, 
                                                                   xlim=self.base_rng.param.x_range,
                                                                   ylim=self.base_rng.param.y_range) *
                                        self.polys)
                                         
                        
                        

                #self.all_maps = self.cov_map.apply.opts(visible=self.cov_viz.param.value) * self.polys

                #self.poly_stream = streams.PolyDraw(source=self.polys, drag=True, num_objects=2,
                #                        show_vertices=True, styles=self.poly_opts)

                return self.all_maps
            else:
                print('test')
                return pn.Column(self.tiles * self.labels)
        else:
            return pn.Column(self.tiles * self.labels)

    def update_stats(self, data):
        if data is None:
            self.bm_gauge.value = 0
        elif len(data['xs']) == 0:
            self.bm_gauge.value = 0
        else:
            if len(data['xs'][0]) < 3:
                self.bm_gauge.value = 0
                self.cov_pie = {
                    'title': {
                        'text': "Cover",
                        'subtext': "Fractional vegetation cover (%)",
                        'left': "center"},
                    'tooltip': {'show': True,
                                'formatter': '{b}: {d} %'}, 
                    'series': [{
                        'type': 'pie',
                        'data': [{'value': 0, 'name': 'Litter'},
                                 {'value': 0, 'name': 'Bare ground'},
                                 {'value': 0, 'name': 'Green veg'},
                                 {'value': 0, 'name': 'Dry veg'}],
                        'color': ['#ee6666', '#91cc75', '#5470c6', '#fac858'],
                        'roseType': 'radius',
                        'radius': ["0%", "45%"],
                        'label': {
                            'edgeDistance': "1%",
                            'bleedMargin': 10,
                            'alignTo': "edge"},
                        'labelLine': {'length': 5}}]}
                
                self.thresh_bar = {
                    'title': {
                        'text': "Biomass threshold",
                        'left': "center"},
                    'tooltip': {},
                    'legend': {},
                    'grid': {'left': '20%'},
                    'xAxis': {
                        'type': 'category',
                        'name': 'Probability of biomass less than ' + str(self.thresh_picker.value) + 'kg/ha',
                        'nameLocation': 'middle',
                        'nameGap': 30,
                        'data': self.thresh_labels},
                    'yAxis': {'max': 1000,
                              'axisLabel': {'formatter': '{value} ha'}},
                    'series': [{
                        'name': "",
                        'type': "bar",
                        'data': [0 for x in self.thresh_labels],
                        'colorBy': "data"}]}
            else:
                polys_tmp = gpd.GeoDataFrame(data=data)
                polys_tmp.set_geometry(polys_tmp.apply(lambda row: Polygon(zip(row['xs'], row['ys'])), axis=1), inplace=True)
                polys_tmp.set_crs(epsg='3857', inplace=True)
                polys_info = polys_tmp[['line_color', 'geometry']].reset_index(drop=True).reset_index().rename(columns={'index': 'id'})
                polys_mask_shp = [(row.geometry, row.id+1) for _, row in polys_info.iterrows()]
                polys_mask = shp2mask(shp=polys_mask_shp, 
                                     transform=self.bm_map[:].data['Biomass'].rio.transform(), 
                                     outshape=self.bm_map[:].data['Biomass'].shape, 
                                     xr_object=self.bm_map[:].data['Biomass'])
                poly_mask_tmp = polys_mask.where(polys_mask == 1, drop=True)
                bm_dat_tmp = self.bm_map[:].data.sel(x=poly_mask_tmp['x'], y=poly_mask_tmp['y'], method='nearest', tolerance=30)
                ds_poly_tmp = self.ds.sel(x=poly_mask_tmp['x'], y=poly_mask_tmp['y'], method='nearest', tolerance=30)
                if not bm_dat_tmp.isnull().all()['Biomass']:
                    bm_hist_dat = bm_dat_tmp.groupby_bins('Biomass', bins=self.bm_plotrange, include_lowest=True,
                                    labels=self.bm_plotrange[1:]).count() / bm_dat_tmp.count() * 100
                    bm_hist_dat = bm_hist_dat.fillna(0)
                    self.bm_gauge.value = int(bm_dat_tmp.mean()['Biomass'])

                    thresh_dat_tmp = self.thresh_map[:].data.sel(x=poly_mask_tmp['x'], y=poly_mask_tmp['y'], method='nearest', tolerance=30)
                    thresh_hist_dat = thresh_dat_tmp.groupby_bins('Threshold', bins=self.thresh_plotrange, include_lowest=True,
                                    labels=self.thresh_labels).count() * 0.90
                    thresh_hist_dat = thresh_hist_dat.fillna(0)

                    da_cov_tmp = self.cov_map[:].data.sel(x=poly_mask_tmp['x'], y=poly_mask_tmp['y'], method='nearest', tolerance=30)
                    cov_factors = [k for k in self.cov_map[:].data.keys() if k not in ['y', 'x']]
                    cov_labels = [self.cov_dict[i] for i in cov_factors]
                    cov_labels.append('Litter')
                    cov_vals = [int(round(float(da_cov_tmp[f].mean())*100, 0)) for f in cov_factors]
                    cov_vals.append(int(round(100 - np.sum(cov_vals), 0)))
                    
                    self.bm_hist = {
                        'title': {
                            'text': 'Biomass variability',
                            'subtext': str(bm_hist_dat['Biomass'].cumsum()[
                                    bm_hist_dat['Biomass_bins'] == self.thresh_picker.value].astype('int').values[0]) +
                            '% of the area is less than the threshold of ' + 
                            str(self.thresh_picker.value) + ' kg/ha',
                            'left': "center"},
                        'tooltip': {},
                        'legend': {},
                        'grid': {'show': False},
                        'xAxis': {
                            'type': 'value',
                            'name': 'Biomass (kg/ha)',
                            'nameLocation': 'middle',
                            'nameGap': 30,
                            'splitLine': {
                                      'show': False}},
                        'yAxis': {'max': round(max(bm_hist_dat.Biomass.values)*1.35, 0),
                                  'axisLabel': {'formatter': '{value} %'},
                                  'splitLine': {
                                      'show': False}},
                        'series': [{
                            'name': "",
                            'type': "bar",
                            'data': [{'value': [bm_hist_dat.Biomass_bins.values[idx], x], 
                                      'itemStyle': {'color': self.bm_colors[idx]}} for idx, x in enumerate(
                                np.round(bm_hist_dat.Biomass.values, 1))],
                            'colorBy': "data",
                            'markLine': {
                                'silent': True,
                                'data': [
                                    [{'coord': [self.thresh_picker.value, 0],
                                      'lineStyle': {
                                          'color': 'black'}}, 
                                     {'coord': [self.thresh_picker.value, round(max(bm_hist_dat.Biomass.values)*1.15, 0)],
                                      'symbol': 'none'}]],
                                'lineStyle': {'color': 'black'},
                                'label': {'formatter': str(self.thresh_picker.value) + ' kg/ha', 
                                          'distance': 10,
                                          'color': 'black'}}}]}
                    self.bm_echart.object = self.bm_hist

                    self.thresh_bar = {
                        'title': {
                            'text': "Biomass threshold",
                            'subtext': 'Area in each category (' + str(int(bm_dat_tmp.count()['Biomass']*0.90)) + 'ha total)', 
                            'left': "center"},
                        'tooltip': {},
                        'legend': {},
                        'grid': {'left': '20%'},
                        'xAxis': {
                            'type': 'category',
                            'name': 'Probability of biomass less than ' + str(self.thresh_picker.value) + 'kg/ha',
                            'nameLocation': 'middle',
                            'nameGap': 30,
                            'data': thresh_hist_dat.Threshold_bins.values},
                        'yAxis': {'max': round(max(thresh_hist_dat.Threshold.values)*1.10, -1),
                                  'axisLabel': {'formatter': '{value} ha'}},
                        'series': [{
                            'name': "",
                            'type': "bar",
                            'data': [{'value': x, 'itemStyle': {'color': self.thresh_colors[idx]}} for idx, x in enumerate(
                                np.round(thresh_hist_dat.Threshold.values, -1))],
                            'colorBy': "data"
                        }],
                    }
                    self.thresh_echart.object = self.thresh_bar

                    self.cov_pie = {
                        'title': {
                            'text': "Cover",
                            'subtext': "Fractional vegetation cover (%)",
                            'left': "center"},
                        'tooltip': {'show': True,
                                    'formatter': '{b}: {d} %'}, 
                        'series': [{
                            'type': 'pie',
                            'data': [{'value': cov_vals[0], 'name': cov_labels[0]},
                                     {'value': cov_vals[1], 'name': cov_labels[1]},
                                     {'value': cov_vals[2], 'name': cov_labels[2]},
                                     {'value': cov_vals[3], 'name': cov_labels[3]}],
                            'color': ['#ee6666', '#91cc75', '#5470c6', '#fac858'],
                            'roseType': 'radius',
                            'radius': ["0%", "45%"],
                            'label': {
                                'edgeDistance': "1%",
                                'bleedMargin': 10,
                                'alignTo': "edge"},
                            'labelLine': {'length': 5}}]}

                    self.cov_echart.object = self.cov_pie
                    
                    self.ts_bm = {
                        'title': [{'left': 'center', 'text': 'Biomass'}],
                        'legend': {'orient': 'vertical',
                                   'right': 10,
                                   'top': 'center'
                                  },
                        'tooltip': {'trigger': 'axis'},
                        'xAxis': [{'data': self.ds['date'].astype('str').values}],
                        'yAxis': [{}],
                        'series': [{'name': 'Region',
                                    'type': 'line',
                                    'showSymbol': False,
                                    'data': self.ds.mean(['y', 'x'])['Biomass'].astype('int').values,
                                    'itemStyle': {'color': 'rgb(0, 0, 0)'}},
                                  {'name': 'Pasture',
                                    'type': 'line',
                                    'showSymbol': False,
                                    'data': ds_poly_tmp.mean(['y', 'x'])['Biomass'].astype('int').values,
                                    'itemStyle': {'color': '#1b9e77'}}]}
                    self.ts_bm_echart.object = self.ts_bm
                    
                    self.ts_ndvi = {
                        'title': [{'left': 'center', 'text': 'Greenness (NDVI)'}],
                        'tooltip': {'trigger': 'axis'},
                        'xAxis': [{'data': self.ds['date'].astype('str').values}],
                        'yAxis': [{}],
                        'series': [{'name': 'Region',
                                    'type': 'line',
                                    'showSymbol': False,
                                    'data': self.ds.mean(['y', 'x'])['NDVI'].values,
                                    'itemStyle': {'color': 'rgb(0, 0, 0)'}},
                                  {'name': 'Pasture',
                                    'type': 'line',
                                    'showSymbol': False,
                                    'data': ds_poly_tmp.mean(['y', 'x'])['NDVI'].values,
                                    'itemStyle': {'color': '#1b9e77'}}]}
                    self.ts_ndvi_echart.object = self.ts_ndvi

                    
    
    @pm.depends('thresh_picker.value', watch=True)
    def trigger_thresh(self):
        if not self.poly_stream.data is None:
            if len(self.poly_stream.data['xs']) > 0:
                self.update_stats(data=self.poly_stream.data)
    
    @pm.depends('viz_sel.value')
    def update_colorbar(self):
        if self.viz_sel.value == 'Basemap':
            return pn.Spacer(height=20, width=self.control_width)
        elif self.viz_sel.value == 'Biomass probability':
            return self.make_colorbar(self.viz_sel.value).options(color_levels=5)
        else:
            return self.make_colorbar(self.viz_sel.value)
    
    def _create_view(self):
        layout = pn.Column(pn.Row(pn.Column(self.year_picker, 
                                            self.date, 
                                            self.controls,
                                            width=int(self.control_width*1.2)), 
                                  pn.Column(self.create_maps),
                                 pn.Tabs(('Single-date', pn.Column(self.bm_gauge,
                                                                   self.bm_echart, 
                                                                   self.thresh_echart, 
                                                                   self.cov_echart, scroll=True,
                                           max_height=650, min_width=int(self.stats_width*1.1))),
                                         ('Time-series', pn.Column(self.ts_bm_echart,
                                                                   self.ts_ndvi_echart)))))
        return layout
                        

In [ ]:
app = getData()
port = 5432
app_served = pn.serve(app.view.servable(),
        websocket_origin='ceres-ood.scinet.usda.gov', port=port, show=False, verbose=False)
url = 'https://ceres-ood.scinet.usda.gov' + jupServer['base_url'] + 'proxy/' + str(port) + '/'
print('\n\nApp launched and ready! Click the link below to run the app: \n\n' + url)
#app.view.servable()

In [ ]:
app_served.stop()

In [ ]:
app.ts_bm

In [ ]:
polys_tmp = gpd.GeoDataFrame(data=app.poly_stream.data)
polys_tmp.set_geometry(polys_tmp.apply(lambda row: Polygon(zip(row['xs'], row['ys'])), axis=1), inplace=True)
polys_tmp.set_crs(epsg='3857', inplace=True)
polys_info = polys_tmp[['line_color', 'geometry']].reset_index(drop=True).reset_index().rename(columns={'index': 'id'})
polys_mask_shp = [(row.geometry, row.id+1) for _, row in polys_info.iterrows()]
polys_mask = shp2mask(shp=polys_mask_shp, 
                     transform=app.bm_map[:].data['Biomass'].rio.transform(), 
                     outshape=app.bm_map[:].data['Biomass'].shape, 
                     xr_object=app.bm_map[:].data['Biomass'])
poly_mask_tmp = polys_mask.where(polys_mask == 1, drop=True)

In [ ]:
ds_poly_tmp = app.ds.sel(x=poly_mask_tmp['x'], y=poly_mask_tmp['y'], method='nearest', tolerance=30)

In [ ]:
#ds_poly_tmp.date

In [ ]:
from bokeh.models import HoverTool
hover = HoverTool(
    tooltips=[
        ("Date", "@date{%b-%d-%Y}"),
        ("Biomass",  "@Biomass{,f} kg/ha")],
    formatters={"@date":"datetime"}, mode='vline')
ds_poly_tmp.mean(['y', 'x'])['Biomass'].astype('int').hvplot(
    color='black').opts(gridstyle={'ygrid_line_color': 'lightgray',
                                   'ygrid_line_width': 1.5,
                                   'minor_ygrid_line_width': 0.5,
                                   'xgrid_line_color': None,
                                   'minor_ygrid_line_color': None},
                        show_grid=True,
                           toolbar=None,
                       tools=[hover], width=app.stats_width)

In [ ]:
from bokeh.models import HoverTool
hover = HoverTool(
    tooltips=[
        ("Cover type", '@Variable'),
        ("Date", "@date{%b-%d-%Y}"),
        ("value",  "@value")],
    formatters={"@date":"datetime"})
ds_poly_tmp.mean(['y', 'x'])[['BARE', 'GREEN', 'SD', 'LITT']].hvplot(
    color=['blue', 'green', 'red', 'orange']).opts(gridstyle={'ygrid_line_color': 'lightgray',
                                   'ygrid_line_width': 1.5,
                                   'minor_ygrid_line_width': 0.5,
                                   'xgrid_line_color': None,
                                   'minor_ygrid_line_color': None},
                        show_grid=True,
                           toolbar=None,
                       tools=[hover], 
                        width=app.stats_width)

In [ ]:
ds_poly_tmp['date'].values[0].strftime('%b-%d-%Y')

In [ ]:
ts_line = {
  'title': [
    {
      'left': 'center',
      'text': 'Biomass'
    }],
  'tooltip': {
      'trigger': 'axis',
      #'formatter': '{b} <br/> {a}: {c} kg/ha',
  },
  'xAxis': [
    {
        'data': ds_poly_tmp['date'].astype('str').values
    }],
  'yAxis': [
    {}],
  'series': [
    {'name': 'Region avg.',
                                    'type': 'line',
                                    'showSymbol': False,
                                    'data': ds.mean(['y', 'x'])['Biomass'].astype('int').values,
                                    'itemStyle': {'color': 'rgb(0, 0, 0)'}},
                                  {'name': 'Pasture avg.',
                                    'type': 'line',
                                    'showSymbol': False,
                                    'data': ds_poly_tmp.mean(['y', 'x'])['Biomass'].astype('int').values,
                                    'itemStyle': {'color': '#1b9e77'}},
  ]}
ts_echart = pn.pane.ECharts(ts_line, height=250, width=app.stats_width, margin=app.stats_margin)

In [ ]:
ts_echart

In [ ]:
app.poly_stream.event()

In [ ]:
list(zip(ds_poly_tmp['date'].astype('str').values, ds_poly_tmp.mean(['y', 'x'])['Biomass'].astype('int').values))

In [ ]:
app.create_maps()